In [2]:
import pandas as pd
import requests
import datetime

import env

In [3]:
search_base_url = 'https://www.googleapis.com/youtube/v3/search'
videos_base_url = 'https://www.googleapis.com/youtube/v3/videos'

search_params = {
  'key': env.token,
  'part': 'snippet',
  'order': 'viewCount',
  'regionCode': 'KR',
  'type': 'video',
  'maxResults': 20,
  'fields': 'items(id/videoId, snippet(publishedAt,title))'
}
videos_params = {
  'key': env.token,
  'part': 'statistics',
  'fields': 'items/statistics'
}

search_keywords = ['영화', '명장면', '리뷰', '해석', '의미']
columns = ['영상 id', '영화 제목', '검색 키워드', '영상 제목', '조회수', '좋아요수', '댓글 수', '생성된 날짜']

tokenLimit = len(env.token)-1

In [21]:
movieInfo = pd.read_excel('Preprocessed_Data.xlsx').loc[:,['movie_name', 'release']]
movieInfo.columns = ['영화명', '개봉일']
movieInfo = movieInfo.astype({'개봉일': 'str'})
movieNames = list(movieInfo.loc[:, '영화명'])

In [22]:
startIndex = 0
endIndex = len(movieNames)

def get_video_list():
  columns = ['영상 id', '영화 제목', '검색 키워드', '영상 제목', '조회수', '좋아요수', '댓글 수', '생성된 날짜']
  video_list = pd.DataFrame(columns=columns)
  tokenNum = 0

  for movieName in movieNames[startIndex:endIndex]:
    print(movieName, len(video_list))
    for keyword in search_keywords:
      # parameter 설정
      publishedAfter = list(movieInfo[movieInfo['영화명'] == movieName].loc[:, '개봉일'])[0]
      year, month, day = publishedAfter.split('-')
      publishedBefore = (datetime.datetime(int(year), int(month), int(day)) + datetime.timedelta(days=30)).strftime('%Y-%m-%d')
      q = ' '.join([movieName, keyword])

      search_params['q'] = q
      search_params['publishedAfter'] = publishedAfter+'T00:00:00Z'
      search_params['publishedBefore'] = publishedBefore+'T00:00:00Z'

      res_search = requests.get(search_base_url, search_params).json()

      # 네트워크 오류 발생 시
      while 'error' in res_search.keys():
        print('[ERROR] search requests ', res_search)
        tokenNum += 1
        if tokenNum > tokenLimit: 
          print('[Error] token 오늘 할당량 끝났다~~', movieName, keyword)
          return video_list

        print('[HTTPError] token 교체')
        search_params['key'] = env.token[tokenNum]
        res_search = requests.get(search_base_url, search_params).json()

      # 응답 데이터 가공
      if 'items' in res_search.keys():
        for item in res_search['items']:
          result_video_ids = list(video_list.loc[video_list['영화 제목'] == movieName,'영상 id'])

          if item['id']['videoId'] not in result_video_ids:
            videos_params['id'] = item['id']['videoId']
            res_videos = requests.get(videos_base_url, videos_params).json()

            # 네트워크 오류 발생 시
            while 'error' in res_videos.keys():
              print('[ERROR] video requests ', res_videos)
              tokenNum += 1
              if tokenNum > tokenLimit: 
                print('[Error] token 오늘 할당량 끝났다~~', movieName)
                return video_list

              print('[HTTPError] token 교체')
              videos_params['key'] = env.token[tokenNum]
              res_videos = requests.get(videos_base_url, videos_params).json()

            if 'items' in res_videos.keys():
              try:
                video_statistics = res_videos['items'][0]['statistics']
                item_data = [[
                  item['id']['videoId'], 
                  movieName, 
                  q, 
                  item['snippet']['title'], 
                  int(video_statistics['viewCount']) if 'viewCount' in video_statistics else 0,
                  int(video_statistics['likeCount']) if 'likeCount' in video_statistics else 0,
                  int(video_statistics['commentCount']) if 'commentCount' in video_statistics else 0,
                  item['snippet']['publishedAt'], 
                ]]
                video_list = video_list.append(pd.DataFrame(item_data, columns=columns),ignore_index=True)
              except Exception as e:
                print('[ERROR] ', e, movieName, res_videos)
                return video_list


  return video_list

In [23]:
video_list = get_video_list()

23 아이덴티티 0
너의 결혼식 24
기억의 밤 57
어벤져스: 인피니티 워 88
오션스8 143
암수살인 167
탐정: 리턴즈 199
보스 베이비 223
완벽한 타인 245
저스티스 리그 278
분노의 질주: 더 익스트림 314
창궐 336
공작 369
명당 402
에이리언: 커버넌트 432
마녀 461
도어락 488
가디언즈 오브 갤럭시 VOL. 2 518
조작된 도시 562
[ERROR] search requests  {'error': {'code': 403, 'message': 'The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quota">quota</a>.', 'errors': [{'message': 'The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quota">quota</a>.', 'domain': 'youtube.quota', 'reason': 'quotaExceeded'}]}}
[HTTPError] token 교체
서치 597
쥬라기 월드: 폴른 킹덤 624
[ERROR] video requests  {'error': {'code': 403, 'message': 'The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quota">quota</a>.', 'errors': [{'message': 'The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quota">quota</a>.', 'domain': 'youtube.quota', 'reason

In [24]:
result = video_list.set_index('영상 id')

In [25]:
result.to_csv(
  ''.join(['./videoList/movieVideoList_', str(startIndex), '_', str(endIndex-1),'.csv']), 
  encoding='utf-8-sig')

In [26]:
top = lambda x: x.sort_values(by='조회수', ascending=False)[:10]
filtered_result = result.groupby('영화 제목').apply(top)
filtered_result

영화 제목          검색 키워드  \
영화 제목       영상 id                                      
1987        N8lgrc4EMZs         1987         1987 리뷰   
            J2kq3HHipqs         1987         1987 영화   
            2jnN7OVzRo8         1987         1987 리뷰   
            _RRKuucepKI         1987         1987 영화   
            Psg7Wv7K1Yg         1987         1987 영화   
...                              ...             ...   
혹성탈출: 종의 전쟁 Iv4Sm7_wUnI  혹성탈출: 종의 전쟁  혹성탈출: 종의 전쟁 영화   
            fAnbVtReE50  혹성탈출: 종의 전쟁  혹성탈출: 종의 전쟁 영화   
            LiE68X9KuP0  혹성탈출: 종의 전쟁  혹성탈출: 종의 전쟁 영화   
            TcpDrNMekFI  혹성탈출: 종의 전쟁  혹성탈출: 종의 전쟁 영화   
            7i2qmxVSoQg  혹성탈출: 종의 전쟁  혹성탈출: 종의 전쟁 영화   

                                                                     영상 제목  \
영화 제목       영상 id                                                            
1987        N8lgrc4EMZs             1987관람 후 목이 메인 문재인대통령을 보고 결국 오열하는 강동원.   
            J2kq3HHipqs  [단독] 박종철 고문치사 당시 지휘관 &#39;31년 만의 고백&#39; / SBS...   
            2jnN7OVzRo8                                「1987」 공식을 무너뜨린 기발함   
            _RRKuucepKI  상상 이상의 완성도, 한국 영화의 수준을 다시 보게 하다 : 1987 리뷰 - 쇼킹...   
            Psg7Wv7K1Yg  영화 &#39;1987&#39;의 실제 인물에게 듣는 그때의 이야기! &#39;나는...   
...                                                                    ...   
혹성탈출: 종의 전쟁 Iv4Sm7_wUnI                           [혹성탈출:진화의 시작] 노오오오오오오!!!   
            fAnbVtReE50                     [좋거나 혹은 나쁘거나] 03 - 혹성탈출: 종의 전쟁   
            LiE68X9KuP0  혹성탈출 1, 2 스토리 완벽 요약! 이 영상 보고 종의 전쟁 보러 가세요! - 라...   
            TcpDrNMekFI                               [혹성탈출:반격의 서막] 시저와 코바   
            7i2qmxVSoQg                              혹성탈출: 종의 전쟁(혹성탈출3) 리뷰   

                             조회수   좋아요수  댓글 수                생성된 날짜  
영화 제목       영상 id                                                    
1987        N8lgrc4EMZs  1840826  11625  2295  2018-01-07T05:33:58Z  
            J2kq3HHipqs   727895   5953   815  2018-01-23T06:08:42Z  
            2jnN7OVzRo8   598318   4816   771  2017-12-31T01:56:37Z  
            _RRKuucepKI   507279   4862   706  2018-01-17T09:18:06Z  
            Psg7Wv7K1Yg   502998   6574   587  2018-01-12T23:30:01Z  
...                          ...    ...   ...                   ...  
혹성탈출: 종의 전쟁 Iv4Sm7_wUnI    33907    178    32  2017-08-25T06:25:31Z  
            fAnbVtReE50    29083    309    45  2017-08-29T06:14:18Z  
            LiE68X9KuP0    27377    241    50  2017-08-23T02:25:31Z  
            TcpDrNMekFI    24471    127    16  2017-09-01T01:00:00Z  
            7i2qmxVSoQg    20335    231    68  2017-08-26T09:00:01Z  

[970 rows x 7 columns]

In [27]:
filtered_result.to_csv(
  ''.join(['./filteredVideoList/movieVideoList_',str(startIndex),'_',str(endIndex-1),'.csv']), 
  encoding='utf-8-sig'
)

### 파일 이어 붙이기

In [10]:
import glob
import pandas as pd

In [19]:
filenames = glob.glob('./filteredVideoList/*.csv')
print(filenames)

df_list = (pd.read_csv(f) for f in filenames)
concated_list = pd.concat(df_list, ignore_index=True)
concated_list = concated_list.drop(columns=['영화 제목.1'], axis=1)

['./filteredVideoList/movieVideoList_12_25.csv', './filteredVideoList/movieVideoList_65_94.csv', './filteredVideoList/movieVideoList_0_11.csv', './filteredVideoList/movieVideoList_26_64.csv']


In [22]:
lastVideoList = pd.read_csv('./filteredVideoList/movieVideoList_65_94.csv').drop(columns=['영화 제목.1'], axis=1)
lastVideoList.to_csv('./videoList_final.csv', encoding='utf-8-sig')

,영화 제목,영상 id,검색 키워드,영상 제목,조회수,좋아요수,댓글 수,생성된 날짜
0,검객,D3oecFZmPnM,검객 영화,"역대 최고 중 하나가 된 무협 영화, 진시황의 앞에 섰던 검객의 이야기 [영화리뷰]",850379,5064,744,2020-09-25T07:21:37Z
1,검객,mQxVJZvflBs,검객 영화,"맹인 지팡이 속 숨겨진 칼날의 비밀, 검객 자토이치의 정체 [영화리뷰/결말포함]",684936,5002,1025,2020-10-03T07:51:07Z
2,검객,3pFteU56o5w,검객 영화,아이를 업은 채 적들을 쓸어버리는 몰락한 가문 최강의 검객! [영화리뷰/결말포함],306156,2776,241,2020-10-11T12:06:41Z
3,검객,oE9arnb-krE,검객 영화,신민아와 장혁이 각각 다른 영화로 돌아왔다: 디바 &amp; 검객 미리 보고 온 후기,32575,599,104,2020-09-23T08:00:12Z
4,검객,dBU9XjCn1_0,검객 영화,검객 THE SWORDSMAN on＜KBS2 &#39;영화가좋다 I love mov...,27614,149,22,2020-10-03T02:33:33Z
...,...,...,...,...,...,...,...,...
295,프리즌 이스케이프,xt5v39dwmYQ,프리즌 이스케이프 영화,[영화랑] 70년 남아공 실화 탈옥 스릴러 &#39;프리즌 이스케이프&#39; 역주...,1429,13,0,2020-05-21T01:23:44Z
296,프리즌 이스케이프,p0bcqMnEwUM,프리즌 이스케이프 영화,[프리즌 이스케이프] 다니엘 래드클리프 인터뷰 영상,1316,19,1,2020-05-15T02:43:07Z
297,프리즌 이스케이프,6RRUJ8XjE4E,프리즌 이스케이프 영화,[특별 예고편] 프리즌 이스케이프 #3 탈출,864,4,0,2020-05-08T09:01:56Z
298,프리즌 이스케이프,JTWRNu_fUjE,프리즌 이스케이프 영화,열쇠로만 스릴을 유도하는 쾌감실화! / 프리즌 이스케이프 리뷰,775,8,8,2020-05-08T10:39:33Z


In [20]:
concated_list.to_csv('./videoList_final.csv', encoding='utf-8-sig')